In [39]:
import plotly.graph_objects as go
import json
import pandas as pd

In [40]:
def get_flops_per_joule(files):
    results = {}
    for gpu, fname in files.items():
        with open(fname, "r") as f:
            line = f.readline()
            data = json.loads(line)
            results[gpu] = data
    gpus = []
    flops_per_joule_forward = []
    flops_per_joule_backward = []
    forward_error = []
    backward_error = []
    
    for gpu, data in results.items():
        gpus.append(gpu)
        
        # Calculate FLOPS per joule
        fwd_flops_per_joule = data['forward_flops_sum'] / data['forward_energy_sum']
        bwd_flops_per_joule = data['backward_flops_sum'] / data['backward_energy_sum']
        
        flops_per_joule_forward.append(fwd_flops_per_joule)
        flops_per_joule_backward.append(bwd_flops_per_joule)
        
        # Calculate error bars using SEM for energy (propagate error for FLOPS/energy ratio)
        # For ratio A/B, relative error = sqrt((sem_A/A)^2 + (sem_B/B)^2)
        # Since FLOPS is constant (no error), error comes only from energy SEM
        fwd_rel_error = data['forward_energy_sem'] / data['forward_energy_sum']
        bwd_rel_error = data['backward_energy_sem'] / data['backward_energy_sum']
        
        forward_error.append(fwd_flops_per_joule * fwd_rel_error)
        backward_error.append(bwd_flops_per_joule * bwd_rel_error)
    
    return gpus, flops_per_joule_forward, flops_per_joule_backward, forward_error, backward_error

In [41]:
def get_time_data(files):
    results = {}
    for gpu, fname in files.items():
        with open(fname, "r") as f:
            line = f.readline()
            data = json.loads(line)
            results[gpu] = data
    
    gpus = []
    fwd_cpu_time = []
    bwd_cpu_time = []
    fwd_gpu_time = []
    bwd_gpu_time = []
    fwd_cpu_time_err = []
    bwd_cpu_time_err = []
    fwd_gpu_time_err = []
    bwd_gpu_time_err = []
    
    for gpu, data in results.items():
        gpus.append(gpu)
        
        # Times in microseconds, convert to milliseconds
        fwd_cpu_time.append(data['forward_cpu_time_mean'] / 1000)
        bwd_cpu_time.append(data['backward_cpu_time_mean'] / 1000)
        fwd_gpu_time.append(data['forward_gpu_time_mean'] / 1000)
        bwd_gpu_time.append(data['backward_gpu_time_mean'] / 1000)
        
        # Error bars (SEM) also convert to milliseconds
        fwd_cpu_time_err.append(data['forward_cpu_time_sem'] / 1000)
        bwd_cpu_time_err.append(data['backward_cpu_time_sem'] / 1000)
        fwd_gpu_time_err.append(data['forward_gpu_time_sem'] / 1000)
        bwd_gpu_time_err.append(data['backward_gpu_time_sem'] / 1000)
    
    return (gpus, fwd_cpu_time, bwd_cpu_time, fwd_gpu_time, bwd_gpu_time,
            fwd_cpu_time_err, bwd_cpu_time_err, fwd_gpu_time_err, bwd_gpu_time_err)

In [42]:
# FP32 plot
files_fp32 = {
    "V100": "experiments/gpu_efficiency/results/eff_v100_fp32_llama3.2_1b_energy_results.jsonl",
    "A100": "experiments/gpu_efficiency/results/eff_a100_80g_fp32_llama3.2_1b_energy_results.jsonl",
    "A40":  "experiments/gpu_efficiency/results/eff_a40_fp32_llama3.2_1b_energy_results.jsonl",
    "L40S": "experiments/gpu_efficiency/results/eff_l40_fp32_llama3.2_1b_energy_results.jsonl"
}

gpus, fp32_forward, fp32_backward, fp32_fwd_err, fp32_bwd_err = get_flops_per_joule(files_fp32)
fp32_forward = [x / 1e9 for x in fp32_forward]
fp32_backward = [x / 1e9 for x in fp32_backward]
fp32_fwd_err = [x / 1e9 for x in fp32_fwd_err]
fp32_bwd_err = [x / 1e9 for x in fp32_bwd_err]

fig = go.Figure(data=[
    go.Bar(name='Forward FP32', x=gpus, y=fp32_forward, 
           error_y=dict(type='data', array=fp32_fwd_err, visible=True)),
    go.Bar(name='Backward FP32', x=gpus, y=fp32_backward,
           error_y=dict(type='data', array=fp32_bwd_err, visible=True))
])
fig.update_layout(
    title="GPU Efficiency: GFLOPs per joule (Llama3.2 1B, FP32, 10 warmup steps, 100 steps)",
    xaxis_title="GPU",
    yaxis_title="GFLOPs per joule",
    barmode='group'
)
fig.show()

In [43]:
# FP16 plot
files_fp16 = {
    "V100": "experiments/gpu_efficiency/results/eff_v100_fp16_llama3.2_1b_energy_results.jsonl",
    "A100": "experiments/gpu_efficiency/results/eff_a100_80g_fp16_llama3.2_1b_energy_results.jsonl",
    "A40":  "experiments/gpu_efficiency/results/eff_a40_fp16_llama3.2_1b_energy_results.jsonl",
    "L40S": "experiments/gpu_efficiency/results/eff_l40_fp16_llama3.2_1b_energy_results.jsonl"
}

gpus, fp16_forward, fp16_backward, fp16_fwd_err, fp16_bwd_err = get_flops_per_joule(files_fp16)
fp16_forward = [x / 1e9 for x in fp16_forward]
fp16_backward = [x / 1e9 for x in fp16_backward]
fp16_fwd_err = [x / 1e9 for x in fp16_fwd_err]
fp16_bwd_err = [x / 1e9 for x in fp16_bwd_err]

fig2 = go.Figure(data=[
    go.Bar(name='Forward FP16', x=gpus, y=fp16_forward,
           error_y=dict(type='data', array=fp16_fwd_err, visible=True)),
    go.Bar(name='Backward FP16', x=gpus, y=fp16_backward,
           error_y=dict(type='data', array=fp16_bwd_err, visible=True))
])
fig2.update_layout(
    title="GPU Efficiency: GFLOPs per joule (Llama3.2 1B, FP16, 10 warmup steps, 100 steps)",
    xaxis_title="GPU",
    yaxis_title="GFLOPs per joule",
    barmode='group'
)
fig2.show()

In [44]:
# GPU Time Analysis for GPU Efficiency
# Note: We focus on GPU execution time rather than CPU time because:
# - GPU time measures actual computational work on the GPU hardware
# - CPU time includes host-side overhead (kernel launches, memory transfers, PyTorch overhead)
# - For GPU efficiency analysis, we want to understand how efficiently the GPU cores are utilized

gpus, fp32_fwd_cpu, fp32_bwd_cpu, fp32_fwd_gpu, fp32_bwd_gpu, fp32_fwd_cpu_err, fp32_bwd_cpu_err, fp32_fwd_gpu_err, fp32_bwd_gpu_err = get_time_data(files_fp32)

fig4 = go.Figure(data=[
    go.Bar(name='Forward GPU FP32', x=gpus, y=fp32_fwd_gpu,
           error_y=dict(type='data', array=fp32_fwd_gpu_err, visible=True)),
    go.Bar(name='Backward GPU FP32', x=gpus, y=fp32_bwd_gpu,
           error_y=dict(type='data', array=fp32_bwd_gpu_err, visible=True))
])
fig4.update_layout(
    title="GPU Time: Forward vs Backward (Llama3.2 1B, FP32, 10 warmup steps, 100 steps)",
    xaxis_title="GPU",
    yaxis_title="GPU Time (ms)",
    barmode='group'
)
fig4.show()

In [45]:
# FP16 GPU Time plots
# Focusing on GPU execution time as the key metric for GPU efficiency
gpus_fp16, fp16_fwd_cpu, fp16_bwd_cpu, fp16_fwd_gpu, fp16_bwd_gpu, fp16_fwd_cpu_err, fp16_bwd_cpu_err, fp16_fwd_gpu_err, fp16_bwd_gpu_err = get_time_data(files_fp16)

fig6 = go.Figure(data=[
    go.Bar(name='Forward GPU FP16', x=gpus_fp16, y=fp16_fwd_gpu,
           error_y=dict(type='data', array=fp16_fwd_gpu_err, visible=True)),
    go.Bar(name='Backward GPU FP16', x=gpus_fp16, y=fp16_bwd_gpu,
           error_y=dict(type='data', array=fp16_bwd_gpu_err, visible=True))
])
fig6.update_layout(
    title="GPU Time: Forward vs Backward (Llama3.2 1B, FP16, 10 warmup steps, 100 steps)",
    xaxis_title="GPU",
    yaxis_title="GPU Time (ms)",
    barmode='group'
)
fig6.show()

In [46]:
# Combined FP32 vs FP16 GPU Time Comparison
fig7 = go.Figure(data=[
    go.Bar(name='Forward FP32', x=gpus, y=fp32_fwd_gpu,
           error_y=dict(type='data', array=fp32_fwd_gpu_err, visible=True),
           marker_color='lightblue'),
    go.Bar(name='Forward FP16', x=gpus_fp16, y=fp16_fwd_gpu,
           error_y=dict(type='data', array=fp16_fwd_gpu_err, visible=True),
           marker_color='darkblue'),
    go.Bar(name='Backward FP32', x=gpus, y=fp32_bwd_gpu,
           error_y=dict(type='data', array=fp32_bwd_gpu_err, visible=True),
           marker_color='lightcoral'),
    go.Bar(name='Backward FP16', x=gpus_fp16, y=fp16_bwd_gpu,
           error_y=dict(type='data', array=fp16_bwd_gpu_err, visible=True),
           marker_color='darkred')
])
fig7.update_layout(
    title="GPU Time Comparison: FP32 vs FP16 (Llama3.2 1B, 10 warmup steps, 100 steps)",
    xaxis_title="GPU",
    yaxis_title="GPU Time (ms)",
    barmode='group'
)
fig7.show()

In [47]:
gpus_fp32, fp32_fwd, fp32_bwd, fp32_fwd_err, fp32_bwd_err = get_flops_per_joule(files_fp32)
gpus_fp16, fp16_fwd, fp16_bwd, fp16_fwd_err, fp16_bwd_err = get_flops_per_joule(files_fp16)
assert gpus_fp32 == gpus_fp16, "GPU lists do not match!"

df = pd.DataFrame({
    "GPU":            gpus_fp32,
    "FP32 Forward":   fp32_fwd,
    "FP32 Backward":  fp32_bwd,
    "FP16 Forward":   fp16_fwd,
    "FP16 Backward":  fp16_bwd,
}).set_index("GPU")

df_g = df / 1e9  # Convert to GFLOPs

# Also create error DataFrame
df_err = pd.DataFrame({
    "GPU":            gpus_fp32,
    "FP32 Forward":   fp32_fwd_err,
    "FP32 Backward":  fp32_bwd_err,
    "FP16 Forward":   fp16_fwd_err,
    "FP16 Backward":  fp16_bwd_err,
}).set_index("GPU")

df_err_g = df_err / 1e9  # Convert to GFLOPs

release_year = {
    "V100": 2017,
    "A100": 2020,
    "A40":  2021,
    "L40S": 2023
}

tdp = {
    "V100": 300,
    "A100": 400,
    "A40": 300,
    "L40S": 350
}

mem = {
    "V100": 32,
    "A100": 80,
    "A40": 48,
    "L40S": 48
}

df_g.insert(0, "Year", df_g.index.map(lambda gpu: release_year.get(gpu, "—")))
df_g.insert(1, "TDP (W)", df_g.index.map(lambda gpu: tdp.get(gpu, "—")))
df_g.insert(2, "Mem (GB)", df_g.index.map(lambda gpu: mem.get(gpu, "—")))
df_g = df_g.reset_index()  # Move GPU from index to column

columns = [
    ("", "GPU"),
    ("", "Year"),
    ("", "TDP (W)"),
    ("", "Mem (GB)"),
    ("FP32", "Forward"),
    ("FP32", "Backward"),
    ("FP16", "Forward"),
    ("FP16", "Backward")
]
df_g.columns = pd.MultiIndex.from_tuples(columns)

format_dict = {
    ("", "GPU"): "{}",
    ("", "Year"): "{:d}",
    ("", "TDP (W)"): "{:d}",
    ("", "Mem (GB)"): "{:d}",
    ("FP32", "Forward"): "{:,.2f}",
    ("FP32", "Backward"): "{:,.2f}",
    ("FP16", "Forward"): "{:,.2f}",
    ("FP16", "Backward"): "{:,.2f}"
}

def add_simple_borders(styler):
    # Add thick horizontal line below header
    styler.set_table_styles({
        (col[0], col[1]): [
            {'selector': 'th', 'props': [('border-bottom', '2px solid #444 !important')]}
        ] for col in styler.data.columns
    }, overwrite=False)
    # Add vertical border after 'GPU'
    styler.set_table_styles({
        ('', 'GPU'): [
            {'selector': 'td', 'props': [('border-right', '2px solid #444 !important')]},
            {'selector': 'th', 'props': [('border-right', '2px solid #444 !important')]}
        ]
    }, overwrite=False)
    # Add vertical border after 'Mem (GB)'
    styler.set_table_styles({
        ('', 'Mem (GB)'): [
            {'selector': 'td', 'props': [('border-right', '2px solid #444 !important')]},
            {'selector': 'th', 'props': [('border-right', '2px solid #444 !important')]}
        ]
    }, overwrite=False)
    return styler

styled = (
    df_g.style
        .format(format_dict, na_rep="NA")
        .set_table_styles(
            [
                {"selector":"th", "props":[("text-align","center")]},
                {"selector":"td", "props":[("text-align","right")]}],
            overwrite=False
        )
)

styled = add_simple_borders(styled)
styled = styled.hide(axis="index")

print("GPU Efficiency (GFLOPs/Joule)")
print(df_g)
print()
print("Error (GFLOPs/Joule)")
print(df_err_g)
display(styled)

GPU Efficiency (GFLOPs/Joule)
                                    FP32                FP16          
    GPU  Year TDP (W) Mem (GB)   Forward  Backward   Forward  Backward
0  V100  2017     300       32  0.724375  0.405051  0.797050  0.253625
1  A100  2020     400       80  0.849340  0.533988  2.623557  0.874169
2   A40  2021     300       48  1.316697  0.573644  1.957099  0.839090
3  L40S  2023     350       48  2.251809  0.712677  3.944471  1.612235

Error (GFLOPs/Joule)
      FP32 Forward  FP32 Backward  FP16 Forward  FP16 Backward
GPU                                                           
V100      0.000195       0.000019      0.000092       0.000006
A100      0.000283       0.000083      0.062540       0.000175
A40       0.000294       0.000017      0.003725       0.000941
L40S      0.000852       0.000034      0.017024       0.000151


In [48]:
# Display the table sorted by Year descending
sorted_df = df_g.sort_values(('','Year'), ascending=False)

styled_sorted = (
    sorted_df.style
        .format(format_dict, na_rep="NA")
        .set_table_styles(
            [
                {"selector":"th", "props":[("text-align","center")]},
                {"selector":"td", "props":[("text-align","right")]}],
            overwrite=False
        )
)
styled_sorted = add_simple_borders(styled_sorted)
styled_sorted = styled_sorted.hide(axis="index")
print("GPU Efficiency (GFLOPs/Joule) - Sorted by Year")
display(styled_sorted)


GPU Efficiency (GFLOPs/Joule) - Sorted by Year
